In [26]:
!pip install "git+https://github.com/jemisjoky/TorchMPS.git"
!pip install "git+https://github.com/rballester/tntorch.git"
import tntorch as tn
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
!pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jemisjoky/TorchMPS.git to c:\users\piche\appdata\local\temp\pip-req-build-yp294x_2
  Resolved https://github.com/jemisjoky/TorchMPS.git to commit 6c0bc1a8e2c15acba8570ca9ffe2b4a0c7135165
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/jemisjoky/TorchMPS.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-yp294x_2'


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/rballester/tntorch.git to c:\users\piche\appdata\local\temp\pip-req-build-6t_yokjc
  Resolved https://github.com/rballester/tntorch.git to commit 241bf7ad2b806f6677a5e23534247f35f3a70f10
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/rballester/tntorch.git 'C:\Users\piche\AppData\Local\Temp\pip-req-build-6t_yokjc'


Defaulting to user installation because normal site-packages is not writeable


We shall choose the Hyperparameters in our setup

In [27]:
# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')


# Data hyperparameters
nb_train = 20
nb_test = 5
chosen_bs = 1

# Student training parameters
# mps parameters
bond_dim = 20
adaptive_mode = False
periodic_bc = False
# gaussian noise parameters
gn_var = 0.3
gn_mean = 0
n_epochs_lmps = 18
mps_learn_rate = 0.0001
mps_reg = 0.0
student_loss = nn.CrossEntropyLoss().to(chosen_device)

In [28]:
# Import the mnist dataset
train_set = torchvision.datasets.MNIST(root = './datasets', train = True,    transform = transforms.ToTensor(),  download = True )

train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train))

train_iterator = torch.utils.data.DataLoader(dataset = train_set, sampler = train_subset, batch_size=chosen_bs)


test_set = torchvision.datasets.MNIST(root = './datasets', train = False, transform = transforms.ToTensor(),  download = True)

test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test))

test_iterator = torch.utils.data.DataLoader(dataset = test_set, sampler = test_subset, batch_size = chosen_bs)

In [43]:

class LMPS_patcher(nn.Module):
    def __init__(self):
        super(LMPS_patcher, self).__init__()
        self.mps_student = MPS(
            input_dim = 16,
            feature_dim = 2,
            output_dim = 1,
            bond_dim=bond_dim,
            adaptive_mode=adaptive_mode,
            periodic_bc=periodic_bc,
        )
        # our original Φ, which creates the multilinear feature space
        self.feature_map = lambda x : torch.tensor([1, x]).reshape(-1)
        # to make the mps use our feature map for the inputs
        self.mps_student.register_feature_map(self.feature_map)
        # turns the input image into a set of images
        self.folding_params = dict(kernel_size=(4, 4), 
            dilation=1, padding=0, stride=4)
        self.unfolder = torch.nn.Unfold(**self.folding_params)
        nb_patches = 49
        self.lin = nn.Linear(nb_patches, 10)
        self.relu = nn.ReLU()
    def forward(self, x):
        patches = self.unfolder(x)
        nb_patches = 49
        vector = torch.zeros([chosen_bs, 49])
        for i in range(chosen_bs):
            for j in range(nb_patches):
                flattened_patch = patches[i,:,j].reshape((1, 16)) # get patch tensor
                vector[0,i] = self.mps_student(flattened_patch)
        y = self.lin(vector)
        y = self.relu(y)
        return y

student = LMPS_patcher()

# Instantiate the optimizer and softmax
student_optimizer = torch.optim.Adam(student.parameters(), lr = mps_learn_rate,
                                  weight_decay = mps_reg)
softmax = nn.Softmax(dim=1)

# Training loop 
for epoch in range(n_epochs_lmps):
    for (x_mb, y_mb) in train_iterator:
        # Reshape the train_tuple and put on the chosen device
        x_mb = x_mb.to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Foward propagation
        y_hat_mb = student(x_mb)
        loss = student_loss(y_hat_mb, y_mb)
        # Backpropagation
        loss.backward()
        student_optimizer.step()
        student_optimizer.zero_grad()

    print(loss.item())


# get accuracy
for (x_mb, y_mb) in test_iterator:
    print("testing student")
    print(MulticlassAccuracy(num_classes=10)(student(x_mb), y_mb))


torch.Size([2])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 1, 28, 28])
torch.Size([1, 16, 49])
torch.Size([1, 

KeyboardInterrupt: 